# Huffman Coding – Homework

This notebook is your homework assignment on the topic of **Huffman coding**.




## Submission Instructions

Your final work must be submitted on **your personal GitHub account**, in a **public repository** dedicated to this assignment.

Your repository must include:

- the completed and cleanly executed notebook,
- any additional files you created for testing or analysis,
- a short `README.md` explaining how to run your notebook.

The deadline for submission is **December 15** (included).  
Any repository created or updated after this date will be considered late.



## Assignment Overview

Start by reading the wiki page on [Huffman coding](https://en.wikipedia.org/wiki/Huffman_coding) to understand the basic concepts.

**Objectives**:

- Compute character frequencies in a text.
- Build a Huffman tree using a min-heap.
- Derive binary codes from a Huffman tree.
- Implement encoding and decoding functions.

Read each text cell carefully, then complete the corresponding code cells.
You can run the test cells to check your work step by step.

## Exercise 1 – Character frequencies

We first need a function that counts how often each character appears in a text.

- The function should **ignore case**: treat `"A"` and `"a"` as the same character.
- It should return a **dictionary** whose keys are characters and whose values are their frequencies.

> Example:  
> `frequency("abA!")` could return `{ "a": 2, "b": 1, "!": 1 }` (the order of keys does not matter).


In [ ]:
def frequency_manual(text: str) -> dict[str, int]:
    text = text.lower()       # ignore case
    freqs: dict[str, int] = {}     # empty dictionary to store counts
    for ch in text:     # increment the count for this character default 0 if not present yet
        freqs[ch] = freqs.get(ch, 0) + 1
    return freqs
frequency_manual("aaaZERbaidjan")

{'a': 5, 'z': 1, 'e': 1, 'r': 1, 'b': 1, 'i': 1, 'd': 1, 'j': 1, 'n': 1}

or we can use another method with the collections


In [46]:
from collections import Counter
def frequency(text: str) -> dict[str, int]:
    return dict(Counter(text.lower()))
frequency("abA!")

{'a': 2, 'b': 1, '!': 1}

In [47]:
# === Tests for Exercise 1 ===

assert frequency("") == {}
assert frequency_manual("aaa") == {"a": 3}
assert frequency("abA!") == {"a": 2, "b": 1, "!": 1}

print("Exercise 1 seems OK (at least for these simple tests).")

Exercise 1 seems OK (at least for these simple tests).


## Exercise 2 – A node for the Huffman tree

We represent a Huffman tree using a `Node` class.

Each node must store:

- `characters`: a string containing all characters that appear in the subtree rooted at this node;
- `frequency`: the sum of the frequencies of these characters;
- `left`: the left child (another `Node`, or `None` for a leaf);
- `right`: the right child (another `Node`, or `None` for a leaf).

We will also define the method `__lt__` so that nodes can be compared by frequency when we use them in a **min-heap**.


In [ ]:
class Node:
    def __init__(self, characters: str, frequency: int):

        self.characters = characters
        self.frequency = frequency
        self.left = None
        self.right = None

    def __lt__(self, other: "Node") -> bool:
        """Allow comparison of nodes by frequency for use in a min-heap."""
        return self.frequency < other.frequency


In [49]:
# === Tests for Exercise 2 ===

n1 = Node("a", 5)
n2 = Node("b", 3)

# Check attributes exist
assert hasattr(n1, "characters")
assert hasattr(n1, "frequency")
assert hasattr(n1, "left")
assert hasattr(n1, "right")

# Check initial values
assert n1.characters == "a"
assert n1.frequency == 5
assert n1.left is None and n1.right is None

# Check comparison works (this must not crash)
nodes = [n1, n2]
from heapq import heapify
heapify(nodes)

print("Exercise 2 seems OK (basic tests passed).")


Exercise 2 seems OK (basic tests passed).


## Exercise 3 – Building the Huffman tree with a min-heap

We now build the Huffman tree from a dictionary of character frequencies.

**Algorithm (Huffman’s algorithm):**

1. Create a leaf `Node` for each character and push all these nodes into a **min-heap**.
2. While the heap contains more than one node:
   1. Pop the two nodes with the smallest frequency.
   2. Create a new internal node whose:
      - `left` child is the first node,
      - `right` child is the second node,
      - `characters` is the concatenation of the children’s `characters`,
      - `frequency` is the sum of the children’s `frequency`.
   3. Push this new node back into the heap.
3. The last remaining node in the heap is the **root of the Huffman tree**.


In [ ]:
from heapq import heappush, heappop, heapify

def huffman_tree(freqs: dict[str, int]) -> Node:
    # Create one leaf node per character
    heap: list[Node] = [Node(ch, freq) for ch, freq in freqs.items()]
    if not heap:
        # The algorithm does not make sense on an empty frequency table
        raise ValueError("Cannot build Huffman tree from an empty frequency dictionary.")
    # Turn the list into a min-heap ordered by node.frequency 
    heapify(heap)
    
    while len(heap)>1:
        n1 = heappop(heap)
        n2 = heappop(heap)
        parent = Node(n1.characters + n2.characters, n1.frequency + n2.frequency)
        parent.left = n1
        parent.right = n2
        # Push the parent back into the heap
        heappush(heap, parent)
    # The last remaining node is the root of the Huffman tree
    return heap[0]

In [51]:
# === Tests for Exercise 3 ===

freqs = frequency("this is an example for huffman encoding")
root = huffman_tree(freqs)

# Basic structural checks
assert isinstance(root, Node)
assert set(root.characters) == set(freqs.keys())
assert root.frequency == sum(freqs.values())

print("Exercise 3 seems OK (basic tests passed).")


Exercise 3 seems OK (basic tests passed).


## Exercise 4 – Code of a single character

We now want a function that returns the **binary code** of a given character,
based on the Huffman tree.

We assume that:

- a left edge corresponds to the bit `'0'`,
- a right edge corresponds to the bit `'1'`.

**Specification:**

- If `char` does **not** appear in the tree, raise a `ValueError`.
- Otherwise, return a string of `'0'` and `'1'` representing the path from the root to the leaf that contains `char`.
- You may implement this function **recursively**.


In [ ]:
def get_code(tree: Node, char: str) -> str:
    def dfs(node:Node,path:str):
        if node is None:
            # Reached an empty child: stop exploring this branch
            return None
        
        # If this is a leaf node
        if node.left is None and node.right is None:
            # Check if this leaf contains the character we are looking
            if node.characters == char:
                return path or "0"
            # Leaf but not the right character
            return None
        # Try left subtree, adding '0' to the path
        left_code = dfs(node.left,path+"0")
        if left_code is not None:
            return left_code
        # Otherwise try right subtree, adding '1' to the path
        return dfs(node.right,path+"1")
    # Start search from the root with an empty path
    code = dfs(tree,"")
    if code is None:
        # Character does not appear in this tree
        raise ValueError(f"Character {char} not present in the Huffman tree.")
    return code


In [53]:
# === Tests for Exercise 4 ===

freqs = frequency("aaaabbc")
root = huffman_tree(freqs)

code_a = get_code(root, "a")
code_b = get_code(root, "b")
code_c = get_code(root, "c")

# Codes must be non-empty and different for different characters
assert isinstance(code_a, str) and len(code_a) > 0
assert isinstance(code_b, str) and len(code_b) > 0
assert isinstance(code_c, str) and len(code_c) > 0

assert code_a != code_b or code_a != code_c or code_b != code_c

print("Exercise 4 seems OK (basic tests passed).")


Exercise 4 seems OK (basic tests passed).


## Exercise 5 – Listing all codes

We now want to **print all Huffman codes** for all characters in the tree.

Complete the function `show_all_codes(tree)` so that it prints,
for each character in the tree, a line of the form:

```text
Character: x Code: 0101
```

You should reuse your function `get_code` from the previous exercise.


In [ ]:
def show_all_codes(tree: Node) -> None:

    def dfs(node: Node, path: str) -> None:
        if node.left is None and node.right is None:
            # path or '0' handles the special case of a single-node tree
            print(f"{repr(node.characters)}: {path or '0'}")
        else:
            # Traverse left child with '0' added to the path
            if node.left is not None:
                dfs(node.left, path + "0")
            # Traverse right child with '1' added to the path
            if node.right is not None:
                dfs(node.right, path + "1")

    dfs(tree, "")

In [55]:
# === Manual test for Exercise 5 ===

freqs = frequency("this is an example for huffman encoding")
root = huffman_tree(freqs)

# This should print one line per distinct character
show_all_codes(root)


'n': 000
's': 0010
'm': 0011
'h': 0100
't': 01010
'd': 01011
'r': 01100
'l': 01101
'x': 01110
'c': 01111
'p': 10000
'g': 10001
'i': 1001
' ': 101
'u': 11000
'o': 11001
'f': 1101
'e': 1110
'a': 1111


## Exercise 6 – Encoding and decoding

We now implement the two main operations:

- `huffman_encode(text, tree)` returns a **string of bits** (a string of `'0'` and `'1'`),
- `huffman_decode(encoded_text, tree)` returns the original text.

### 6.1 – Encoding

To encode a text:

1. For each character `c` in `text`, look up its code with `get_code(tree, c)`.
2. Concatenate all these codes into a single string.

If a character of `text` does not appear in the tree, you may raise a `ValueError`.


In [ ]:
def huffman_encode(text: str, tree: Node) -> str:
    
    bits: list[str] = []
    for ch in text.lower():
        # get_code returns the bitstring for this character
        bits.append(get_code(tree, ch))
    # Join all partial bitstrings into one
    return "".join(bits)

### 6.2 – Decoding

To decode a string of bits:

1. Start at the **root** of the tree.
2. For each bit in the encoded string:
   - if the bit is `'0'`, go to the **left** child;
   - if the bit is `'1'`, go to the **right** child.
3. Each time you reach a **leaf** (a node whose `left` and `right` are `None`):
   - append its character to the decoded text,
   - go back to the root.

At the end, you should have reconstructed the original text.


In [ ]:
def huffman_decode(encoded_text: str, tree: Node) -> str:
    result_chars: list[str] = []
    node = tree
    for bit in encoded_text:
        if bit == "0":
            if node.left is None:
                raise ValueError("Invalid encoding: reached a leaf on bit '0'.")
            node = node.left
        elif bit == "1":
            if node.right is None:
                raise ValueError("Invalid encoding: reached a leaf on bit '1'.")
            node = node.right
        else:
            # Any character other than '0' or '1' is invalid in this encoding
            raise ValueError(f"Invalid bit {bit!r} in encoded text.")
        if node.left is None and node.right is None:
            result_chars.append(node.characters)
            node = tree
    return "".join(result_chars)

In [58]:
# === Tests for Exercise 6 ===

text = "this is an example for huffman encoding"
freqs = frequency(text)
root = huffman_tree(freqs)

encoded = huffman_encode(text, root)
decoded = huffman_decode(encoded, root)

print("Encoded text:", encoded[:64] + ("..." if len(encoded) > 64 else ""))
print("Decoded text:", decoded)

assert decoded == text.lower(), "Decoded text does not match original (lowercased)."

print("Exercise 6 seems OK (basic round-trip test passed).")


Encoded text: 0101001001001001010110010010101111100010111100111011110011100000...
Decoded text: this is an example for huffman encoding
Exercise 6 seems OK (basic round-trip test passed).


## Exercise 7 – A “universal” Huffman tree for English

So far, all Huffman trees you built were specialised to a specific text.  
In this exercise, you will construct a **general-purpose Huffman tree for English**.

For that purpose, we provide you with a dictionary `english_frequencies` that gives the approximate frequency of each character in English texts.

In [59]:
freqs_english = {
    " ": 18.0,
    "e": 12.02, "t": 9.10, "a": 8.12, "o": 7.68, "i": 7.31, "n": 6.95,
    "s": 6.28, "r": 6.02, "h": 5.92, "d": 4.32, "l": 3.98, "u": 2.88,
    "c": 2.71, "m": 2.61, "f": 2.30, "y": 2.11, "w": 2.09, "g": 2.03,
    "p": 1.82, "b": 1.49, "v": 1.11, "k": 0.69, "x": 0.17, "q": 0.11,
    "j": 0.10, "z": 0.07
}


Start by building a Huffman tree from this dictionary and test it by encoding and decoding the following text:

```text
"Huffman coding is a data compression algorithm." 
```

Make sure that the decoded text matches the original text!


In [60]:
freqs_english_int = {ch: int(freq * 1000) for ch, freq in freqs_english.items()}
english_tree = huffman_tree(freqs_english_int)

sample_text = "Huffman coding is a data compression algorithm."
sample_clean = "".join(ch for ch in sample_text.lower() if ch in freqs_english)

encoded_sample = huffman_encode(sample_clean, english_tree)
decoded_sample = huffman_decode(encoded_sample, english_tree)

print("Original (cleaned):", sample_clean)
print("Encoded (first 80 bits):", encoded_sample[:80] + ("..." if len(encoded_sample) > 80 else ""))
print("Decoded:", decoded_sample)

assert decoded_sample == sample_clean, "Decoded text does not match the original (cleaned) text."

Original (cleaned): huffman coding is a data compression algorithm
Encoded (first 80 bits): 00010000111111011111011111110100110110000001001101111000011010110011010000101110...
Decoded: huffman coding is a data compression algorithm


Compare the length of the encoded text with the length of the original text (in bits).You can assume that each character in the original text is represented using 8 bits (1 byte) using [ASCII encoding](https://en.wikipedia.org/wiki/ASCII).

In [61]:
sample_texts = [
    "This is a short example.",
    "Huffman coding is a data compression algorithm.",
    "The quick brown fox jumps over the lazy dog."
    ]

for text in sample_texts:
    cleaned = "".join(ch for ch in text.lower() if ch in freqs_english)

    encoded = huffman_encode(cleaned, english_tree)
    ascii_bits = len(cleaned) * 8
    huffman_bits = len(encoded)

    print(f"Text: {text!r}")
    print(f"  cleaned: {cleaned!r}")
    print(f"  ASCII bits:   {ascii_bits}")
    print(f"  Huffman bits: {huffman_bits}")
    print()

Text: 'This is a short example.'
  cleaned: 'this is a short example'
  ASCII bits:   184
  Huffman bits: 97

Text: 'Huffman coding is a data compression algorithm.'
  cleaned: 'huffman coding is a data compression algorithm'
  ASCII bits:   368
  Huffman bits: 199

Text: 'The quick brown fox jumps over the lazy dog.'
  cleaned: 'the quick brown fox jumps over the lazy dog'
  ASCII bits:   344
  Huffman bits: 211

